In [1]:
import os
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import Tool, initialize_agent
from langchain.llms import OpenAI
import langchain
langchain.verbose = True
# LangChain Tracing
from langchain.callbacks import get_callback_manager, set_tracing_callback_manager
from langchain.callbacks.tracers import SharedLangChainTracer
set_tracing_callback_manager()

In [2]:
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=True
)

In [8]:
tracer_session = SharedLangChainTracer().new_session()
print(f"New tracing session: {tracer_session}")

agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
)

POST http://127.0.0.1:5000/sessions, status code: 201, id: 1
New tracing session: TracerSession(id=1, start_time=datetime.datetime(2023, 1, 2, 6, 31, 30, 595741), extra={}, child_runs=[])


> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompts after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current events
Calculator: useful for when you need to answer questions about math

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Who won the US Open men's t

"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [9]:
tracer_session.to_dict()

{'id': 1,
 'start_time': datetime.datetime(2023, 1, 2, 6, 31, 30, 595741),
 'extra': {},
 'child_runs': [{'id': None,
   'start_time': datetime.datetime(2023, 1, 2, 6, 31, 30, 658481),
   'end_time': datetime.datetime(2023, 1, 2, 6, 31, 39, 341211),
   'extra': {},
   'execution_order': 1,
   'serialized': {'name': 'AgentExecutor'},
   'session_id': 1,
   'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
   'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
   'child_runs': [],
   'child_llm_runs': [],
   'child_chain_runs': [{'id': None,
     'start_time': datetime.datetime(2023, 1, 2, 6, 31, 30, 658805),
     'end_time': datetime.datetime(2023, 1, 2, 6, 31, 32, 888295),
     'extra': {},
     'execution_order': 2,
     'serialized': {'name': 'LLMChain'},
     'session_id': 1,
     'inputs': {'input': "Who won the US Open men's tenn

In [10]:
import requests

r = requests.get("http://127.0.0.1:5000/sessions/1")
r.json()

{'id': 1,
 'start_time': 1672669890.595741,
 'extra': {},
 'child_runs': [{'id': 1,
   'start_time': 1672669890.658481,
   'end_time': 1672669899.341211,
   'extra': {},
   'execution_order': 1,
   'serialized': {'name': 'AgentExecutor'},
   'session_id': 1,
   'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
   'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
   'child_runs': [{'id': 4,
     'start_time': 1672669890.658805,
     'end_time': 1672669892.888295,
     'extra': {},
     'execution_order': 2,
     'serialized': {'name': 'LLMChain'},
     'session_id': 1,
     'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
      'agent_scratchpad': '',
      'stop': ['\nObservation: ']},
     'outputs': {'text': ' I need to find out who won the US Open men\'s tennis fin

In [11]:
r = requests.get("http://127.0.0.1:5000/chain-runs/1")
r.json()

{'id': 1,
 'start_time': 1672669890.658481,
 'end_time': 1672669899.341211,
 'extra': {},
 'execution_order': 1,
 'serialized': {'name': 'AgentExecutor'},
 'session_id': 1,
 'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
 'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
 'child_runs': [{'id': 4,
   'start_time': 1672669890.658805,
   'end_time': 1672669892.888295,
   'extra': {},
   'execution_order': 2,
   'serialized': {'name': 'LLMChain'},
   'session_id': 1,
   'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
    'agent_scratchpad': '',
    'stop': ['\nObservation: ']},
   'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: "US Open men\'s

In [12]:
r = requests.get("http://127.0.0.1:5000/llm-runs")
r.json()

[{'id': 1,
  'start_time': '2023-01-02T06:31:37.257128',
  'end_time': '2023-01-02T06:31:37.802902',
  'extra': {},
  'execution_order': 13,
  'serialized': {'name': 'OpenAI'},
  'session_id': 1,
  'prompts': ["You are GPT-3, and you can't do math.\n\nYou can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.\n\nSo we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:\n\nQuestion: ${Question with hard calculation.}\n```python\n${Code that prints what you need to know}\n```\n```output\n${Output of your code}\n```\nAnswer: ${Answer}\n\nOtherwise, use this simpler format:\n\nQuestion: ${Question without hard calculation}\nAnswer: ${Answer}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n\n```python\nprint

In [13]:
r = requests.get("http://127.0.0.1:5000/tool-runs")
r.json()

[{'id': 1,
  'start_time': '2023-01-02T06:31:32.888944',
  'end_time': '2023-01-02T06:31:33.016761',
  'extra': {'color': 'green'},
  'execution_order': 4,
  'serialized': {'name': '<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_en...'},
  'session_id': 1,
  'tool_input': "US Open men's tennis final 2019",
  'output': 'Rafael Nadal won his 19th career Grand Slam title, and his fourth US Open crown, by surviving an all-time comback effort from Daniil ...',
  'action': 'Search',
  'child_runs': [],
  'child_llm_runs': [],
  'child_chain_runs': [],
  'child_tool_runs': []},
 {'id': 2,
  'start_time': '2023-01-02T06:31:34.629831',
  'end_time': '2023-01-02T06:31:35.149898',
  'extra': {'color': 'green'},
  'execution_order': 7,
  'serialized': {'name': '<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_en...'},
  'session_id': 1,
  'tool_input': 'Rafael Nadal age',
  'output': '36 years',
  'action': 'Search',
  'child_runs': [],
  'child_llm_runs': [],
  'child_chain_ru

In [14]:
r = requests.get("http://127.0.0.1:5000/chain-runs/top-level")
r.json()

[{'id': 1,
  'start_time': '2023-01-02T06:31:30.658481',
  'end_time': '2023-01-02T06:31:39.341211',
  'extra': {},
  'execution_order': 1,
  'serialized': {'name': 'AgentExecutor'},
  'session_id': 1,
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
  'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
  'child_runs': [{'id': 4,
    'start_time': '2023-01-02T06:31:30.658805',
    'end_time': '2023-01-02T06:31:32.888295',
    'extra': {},
    'execution_order': 2,
    'serialized': {'name': 'LLMChain'},
    'session_id': 1,
    'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
     'agent_scratchpad': '',
     'stop': ['\nObservation: ']},
    'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to 

In [15]:
r = requests.get("http://127.0.0.1:5000/chain-runs")
r.json()

[{'id': 1,
  'start_time': '2023-01-02T06:31:30.658481',
  'end_time': '2023-01-02T06:31:39.341211',
  'extra': {},
  'execution_order': 1,
  'serialized': {'name': 'AgentExecutor'},
  'session_id': 1,
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
  'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
  'child_runs': [{'id': 4,
    'start_time': '2023-01-02T06:31:30.658805',
    'end_time': '2023-01-02T06:31:32.888295',
    'extra': {},
    'execution_order': 2,
    'serialized': {'name': 'LLMChain'},
    'session_id': 1,
    'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
     'agent_scratchpad': '',
     'stop': ['\nObservation: ']},
    'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to 